In [1]:
import pandas as pd
import sklearn
#taller numero 2



Ajustar un modelo para el conjunto de datos. El conjunto de datos contiene información de una
población con la cual se intenta predecir si puede o no cobrar más de 50.000 dólares anuales.

In [2]:
columnas=['edad','clase_de_trabajo','cantidad_personas','educacion',
            'anios_de_estudio','estado_civil','ocupacion','relaciones','raza',
            'sexo','ganancia_capital','perdida_de_capital','horas_por_semana','pais','ganancia']
# Load the Census dataset
data = pd.read_csv("/home/usm/Descargas/Practica Arboles/datos/adult.data",names=columnas, na_values=['?'] )

display(data)


FileNotFoundError: [Errno 2] File b'/home/usm/Descargas/Practica Arboles/datos/adult.data' does not exist: b'/home/usm/Descargas/Practica Arboles/datos/adult.data'

In [ ]:

#datos faltantes
data=data.dropna(axis=0)
display(data.isnull().sum())

In [ ]:
data_complete=data.drop(labels=['cantidad_personas'],axis=1)
#display(data_complete.describe())
#display(data_complete.corr())
#display(data_complete.dtypes)

#tamaño del conjunto de datos
n_records=len(data_complete)
n_most_50=len(data_complete[data_complete['ganancia']=='>50K'])
#n_most=len(data_complete['ganancia']=='<=50k')

print("total:",str(n_records))
print(">50",n_most_50)

In [ ]:
#pasar las variables categoricas a dummies con funcion ya hecha
def function_dummies(df):
    df_dummies=df
    colnames=df.columns
    for c in colnames:
        if df[c].dtypes=='object':
            col=  pd.get_dummies(df[c])
            df_dummies[c+"_"+col.columns]=col
            df_dummies = df_dummies.drop(labels=[c],axis=1)
    return (df_dummies)

Y=data_complete['ganancia']
X=data_complete.drop(labels=['ganancia'],axis=1)
X_dummies=function_dummies(X)
#muestra los 5 primeros
display(X_dummies.head(5))
Y=Y.str.replace('<=50K','1')
Y=Y.replace('>50K','0')


Separe las poblaciones en entrenamiento y validación en forma aleatoria en 70/30
(genere una semilla aleatoria e indique el valor de la semilla en el software que utilizó).
Indique que cantidad de casos quedaron para cada ambiente.

In [ ]:
from sklearn.model_selection import train_test_split
#random_state=0 quiere decir que guarda el mismo conjunto aleatorio
#test_size da el 30% lo oculta para hacer pruebas y el 70% va hacer de validacion
X_train,X_test,y_train,y_test=train_test_split(X_dummies,Y,test_size=0.3,random_state=0)

print(">50 train", y_train[y_train=='1'].count())
print("<=50 train", y_train[y_train=='0'].count())

#print(">50 train", y_test[y_test=='1'].count())
#print("<=50 train", y_test[y_test=='0'].count())

In [ ]:
# Import train_test_split
from sklearn import tree
#arbol de 4 niveles max_depth=4
modelo=tree.DecisionTreeClassifier(max_depth=4,criterion='entropy')

#entrena la variable
modelo.fit(X_train,y_train)

In [ ]:
from sklearn.externals.six import StringIO 
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image 
#pip install graphviz
#pip install pydotplus
dot_data = StringIO()

feature_cols=X_train.columns
export_graphviz(modelo, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('arbol.png')
Image(graph.create_png())

Calcular la perfomarnce y el AUC y el gráfico ROC en entrenamiento y validación
indicando también el total de casos de cada una de las clases.

In [ ]:
#Esta parte es para validar bajo la metrica
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score

#predict test
y_pred_test=modelo.predict(X_test)
#predict train
#Que tanto predice los datos que miro
y_pred_train=modelo.predict(X_train)

display('Test:',confusion_matrix(y_test,y_pred_test))
tn,fp,fn,tp=confusion_matrix(y_test,y_pred_test).ravel()
print("verdaderos positivos",tp)
print("verdadetos negativos",tn)
print("falsos positivos",fp)
print("falsos negativos",fn)
ac=(tp+tn)/(tp+tn+fn+fp)
print("performance", ac)

#forma resumida de lo anterior
print("accuracy test",accuracy_score(y_test,y_pred_test))
print("accuracy train",accuracy_score(y_train,y_pred_train))



In [ ]:
#genera una matriz de 2*2 puesto que tiene 2 clases
probs_test= modelo.predict_proba(X_test)



data_set_test=X_test.copy()
display(data_set_test.head(5))

#va agregar una columna
data_set_test['predicho']=y_pred_test
data_set_test['predicho_prob_1']=probs_test[:,1]
data_set_test['predicho_prob_0']=probs_test[:,0]
data_set_test['real']=y_test

display(data_set_test.head(10))




Selecciones el 25% de los individuos en el ambiente de validación de acuerdo a la
siguiente lógica. Entregue los resultados indicados:
a. Al azar e indique la cantidad de individuos que cobran más de 50.000 dólares.


In [ ]:
X_test_75, X_25, y_test_75, y_25 = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size = 0.25, 
                                                    random_state = 0)

print ("total 25%",y_25.count())
print(">50k test 25% azar",y_25[y_25=='1'].count())

b. Utilizando el modelo desarrollado en el punto 1.1.2 e indique la cantidad de
individuos que cobran más de 50.000 dólares.

In [ ]:
#usando modelo

y_25_modelo=data_set_test.sort_values(['predicho_prob_1'], ascending=[False])[1:y_25.count()]
print(">50k test 25% modelo",len(y_25_modelo[y_25_modelo['real'] == '1']))